In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('Missing').getOrCreate()

24/05/06 20:58:10 WARN Utils: Your hostname, lenis-Nitro-AN515-44 resolves to a loopback address: 127.0.1.1; using 192.168.10.31 instead (on interface wlp4s0)
24/05/06 20:58:10 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/06 20:58:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
training = spark.read.csv('test1_1.csv',header=True,inferSchema=True)

In [4]:
training.show()

+---------+---+----------+------+
|     Name|age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2| 18000|
+---------+---+----------+------+



In [5]:
training.columns

['Name', 'age', 'Experience', 'Salary']

In [8]:
from pyspark.ml.feature import VectorAssembler
featureassembler = VectorAssembler(inputCols=['age','Experience'],outputCol='Independent Features')

In [9]:
output = featureassembler.transform(training)

In [10]:
output.show()

+---------+---+----------+------+--------------------+
|     Name|age|Experience|Salary|Independent Features|
+---------+---+----------+------+--------------------+
|    Krish| 31|        10| 30000|         [31.0,10.0]|
|Sudhanshu| 30|         8| 25000|          [30.0,8.0]|
|    Sunny| 29|         4| 20000|          [29.0,4.0]|
|     Paul| 24|         3| 20000|          [24.0,3.0]|
|   Harsha| 21|         1| 15000|          [21.0,1.0]|
|  Shubham| 23|         2| 18000|          [23.0,2.0]|
+---------+---+----------+------+--------------------+



In [12]:
output.printSchema()

root
 |-- Name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)
 |-- Independent Features: vector (nullable = true)



In [13]:
finalized_data = output.select("Independent Features","Salary")

In [14]:
finalized_data.show()

+--------------------+------+
|Independent Features|Salary|
+--------------------+------+
|         [31.0,10.0]| 30000|
|          [30.0,8.0]| 25000|
|          [29.0,4.0]| 20000|
|          [24.0,3.0]| 20000|
|          [21.0,1.0]| 15000|
|          [23.0,2.0]| 18000|
+--------------------+------+



In [30]:
from pyspark.ml.regression import LinearRegression

train_data, test_data = finalized_data.randomSplit([0.75,0.25])
regressor = LinearRegression(featuresCol='Independent Features',labelCol='Salary')
regressor = regressor.fit(train_data)

24/05/06 21:10:41 WARN Instrumentation: [58e3ded8] regParam is zero, which might cause numerical instability and overfitting.


In [31]:
regressor.coefficients

DenseVector([-64.8464, 1584.7554])

In [32]:
regressor.intercept

15414.10693970376

In [33]:
test_data.show()

+--------------------+------+
|Independent Features|Salary|
+--------------------+------+
|          [24.0,3.0]| 20000|
+--------------------+------+



In [34]:
#Prediction
pred_results = regressor.evaluate(test_data)

In [35]:
pred_results.predictions.show()

+--------------------+------+-----------------+
|Independent Features|Salary|       prediction|
+--------------------+------+-----------------+
|          [24.0,3.0]| 20000|18612.05915813422|
+--------------------+------+-----------------+



In [36]:
pred_results.meanAbsoluteError,pred_results.meanSquaredError

(1387.9408418657804, 1926379.7805190913)